# Language Model Dataset

In [1]:
import pandas as pd
import numpy as np
import os
import random

In [2]:
dataset = pd.read_csv('./lm_dataset/OpenSubtitles.csv', encoding='utf-8')
sub_dataset = pd.read_csv('./lm_dataset/new_kor.csv', encoding='cp949')
dataset = pd.concat([dataset, sub_dataset])

sub_dataset = pd.read_csv('./lm_dataset/tatoeba.csv', encoding='utf-8')
dataset = pd.concat([dataset, sub_dataset])

sub_dataset = pd.read_csv('./lm_dataset/네이버 해커톤 데이터셋.csv', encoding='utf-8')
dataset = pd.concat([dataset, sub_dataset])

for i in range(10):
    sub_dataset = pd.read_csv('./lm_dataset/AI Hub 한-영 데이터셋[%s].csv' % i, encoding='utf-8')
    dataset = pd.concat([dataset, sub_dataset])

In [3]:
len(dataset)

3048264

In [4]:
dataset.head()

,ko
0,"폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘..."
1,우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!
2,황새 아저씨를 기다리세요
3,찾아와 선물을 주실 거예요
4,가난하든 부자이든 상관이 없답니다


In [5]:
dataset = list(dataset['ko'])
random.shuffle(dataset)
dataset[:5]

['정말요?',
 '국립환경과학원은 “대부분 지역에서 미세먼지 수준이 ‘보통’을 나타내겠으나 일부 수도권은 대기 정체로 미세먼지가 축적돼 농도가 높을 것”이라고 설명했다.',
 '특히 고용 감소가 정규직을 중심으로 나타날 가능성이 높다는 지적이다.',
 '제가 같이 들어 드려도 될까요?',
 '머리카락 타는 냄새']

In [6]:
ch_labels = list()

for data in dataset:
    for ch in data:
        if ch not in ch_labels:
            ch_labels.append(ch)

In [7]:
len(ch_labels)

4882

In [8]:
lm_labels = {'char' : ch_labels}
lm_labels = pd.DataFrame(lm_labels)
lm_labels.to_csv('lm_labels.csv', encoding='utf-8', index=False)

In [9]:
lm_labels

,char
0,정
1,말
2,요
3,?
4,국
...,...
4877,敷
4878,杉
4879,倫
4880,處


In [10]:
ch_labels

['정',
 '말',
 '요',
 '?',
 '국',
 '립',
 '환',
 '경',
 '과',
 '학',
 '원',
 '은',
 ' ',
 '“',
 '대',
 '부',
 '분',
 '지',
 '역',
 '에',
 '서',
 '미',
 '세',
 '먼',
 '수',
 '준',
 '이',
 '‘',
 '보',
 '통',
 '’',
 '을',
 '나',
 '타',
 '내',
 '겠',
 '으',
 '일',
 '도',
 '권',
 '기',
 '체',
 '로',
 '가',
 '축',
 '적',
 '돼',
 '농',
 '높',
 '것',
 '”',
 '라',
 '고',
 '설',
 '명',
 '했',
 '다',
 '.',
 '특',
 '히',
 '용',
 '감',
 '소',
 '규',
 '직',
 '중',
 '심',
 '날',
 '능',
 '성',
 '는',
 '제',
 '같',
 '들',
 '어',
 '드',
 '려',
 '될',
 '까',
 '머',
 '리',
 '카',
 '락',
 '냄',
 '새',
 '만',
 '하',
 '면',
 '애',
 '랑',
 '살',
 '할',
 '거',
 '예',
 '게',
 '러',
 '브',
 '를',
 '누',
 '곳',
 '야',
 '-',
 '우',
 '집',
 '뭔',
 '안',
 '좋',
 '있',
 '그',
 '랬',
 'O',
 'h',
 ',',
 '발',
 '!',
 'c',
 'o',
 'm',
 'e',
 'n',
 'I',
 'v',
 'i',
 's',
 't',
 'd',
 'y',
 'u',
 'p',
 'a',
 'l',
 '왜',
 '렇',
 '해',
 'G',
 '마',
 '켓',
 '매',
 '사',
 '트',
 '7',
 '오',
 '전',
 '9',
 '시',
 '터',
 '1',
 '저',
 '녁',
 '8',
 '5',
 '간',
 '진',
 '행',
 '된',
 '차',
 '자',
 '선',
 '2',
 '천',
 '4',
 '6',
 '입',
 '찰',
 '참',
 '여',
 '촬'

In [11]:
acoustic_labels = list(pd.read_csv('train_labels.csv')['char'])

In [88]:
count = 0

for data in dataset:
    for ch in data:
        if ch not in acoustic_labels:
            count += 1
            continue

In [89]:
count

1639229

In [90]:
max_len = -1

for data in dataset:
    if max_len < len(data):
        max_len = len(data)

print(max_len)

428


In [14]:
def num2kor(sentence, mode = 'formal'):
    def detect(sentence):
        digit_part = list()
        tmp = list()
        start_checked = False
        upper = ['⁰','¹','²','³','⁴','⁵','⁶','⁷','⁸','⁹']
        
        for idx,char in enumerate(sentence):
            if char.isdigit():
                if not start_checked:
                    start_checked = True
                tmp.append(char)
            else:
                if sentence[idx] == ',' and sentence[idx-1].isdigit() and sentence[idx+1].isdigit():
                    tmp.append(char)
                elif sentence[idx] in upper:
                     tmp.append(sentence[idx])
                else:
                    if start_checked:
                        start_checked = False
                        digit_part.append(''.join(tmp))
                        tmp = list()
        digit_part.append(''.join(tmp))
        return digit_part
    
    def read(digit_part, mode):           
        small_scale = ['','십','백','천']
        large_scale = ['','만','억','조','경','해']
        num = ['영','일','이','삼','사','오','육','칠','팔','구']

        upper = ['⁰','¹','²','³','⁴','⁵','⁶','⁷','⁸','⁹']
        has_upper = False
        up_idx = 0
        for digit in digit_part:
            if digit in upper: has_upper = True

        digit_part = digit_part.replace(',','')
        length = len(digit_part)
        
        if has_upper:
            length -= 1
            up = digit_part[-1]
            digit_part = digit_part[:-1]
            for idx, ups in enumerate(upper):
                if ups == up:
                    up_idx = idx

        korean = str()
        tmp = str()
        for idx, digit in enumerate(digit_part):
            inversed_idx = length - idx - 1
            if mode == 'formal':
                if inversed_idx % 4:
                    if int(digit):
                        tmp += num[int(digit)]
                        tmp += small_scale[inversed_idx%4]
                else:
                    if int(digit) or length == 1:
                        tmp += num[int(digit)]
                    if tmp:
                        tmp += large_scale[inversed_idx//4]
                    korean += tmp
                    tmp = str()

            elif mode == 'informal':
                if inversed_idx % 4:
                    if int(digit) > 1:
                        tmp += num[int(digit)]
                    if int(digit):
                        tmp += small_scale[inversed_idx%4]
                else:
                    try:
                        if int(digit) or length == 1:
                            tmp += num[int(digit)]
                    except:
                        continue
                    if tmp:
                        tmp += large_scale[inversed_idx//4]
                    if length == 5 and int(digit) == 1 and inversed_idx == 4:
                        tmp = tmp[1:]
                    korean += tmp
                    tmp = str()
        korean += tmp

        if has_upper:
            korean += '의 '+ num[up_idx] +' 승'

        return korean
    
    target = sentence
    digit_part = detect(target)
    
    #print("Target Sentence : ", sentence)
    #print("Digit Part of This Sentence : ", digit_part)
    for each in digit_part:
        readed = read(each, mode = mode)
        if each != '':
            print(each, ' -> ', readed)
        target = target.replace(each, readed, 1)
        
    return target

In [13]:
new_dataset = list()

for sentence in dataset:
    new_sentence = num2kor(sentence, mode='informal')
    new_dataset.append(new_sentence)

7  ->  칠
9  ->  구
11  ->  십일
8  ->  팔
5  ->  오
1  ->  일
2  ->  이
546  ->  오백사십육
26  ->  이십육
25  ->  이십오
5  ->  오
1  ->  일
24  ->  이십사
2  ->  이
1  ->  일
4  ->  사
10  ->  십
30  ->  삼십
32  ->  삼십이
1  ->  일
1  ->  일
5  ->  오
500  ->  오백
1  ->  일
1000  ->  천
27  ->  이십칠
3  ->  삼
2  ->  이
40  ->  사십
7  ->  칠
1  ->  일
13  ->  십삼
2  ->  이
40  ->  사십
5  ->  오
9  ->  구
2009  ->  이천구
3  ->  삼
4  ->  사
2  ->  이
5,300  ->  오천삼백
8  ->  팔
7,000  ->  칠천
84  ->  팔십사
12  ->  십이
15  ->  십오
10  ->  십
1  ->  일
2000  ->  이천
7  ->  칠
39  ->  삼십구
39  ->  삼십구
2  ->  이
3  ->  삼
1  ->  일
3  ->  삼
31  ->  삼십일
25  ->  이십오
2  ->  이
19  ->  십구
10  ->  십
1  ->  일
2019  ->  이천십구
3  ->  삼
37  ->  삼십칠
29  ->  이십구
1  ->  일
4600  ->  사천육백
1  ->  일
2  ->  이
10  ->  십
1  ->  일
29  ->  이십구
19  ->  십구
7  ->  칠
23  ->  이십삼
1  ->  일
1  ->  일
52  ->  오십이
11  ->  십일
1  ->  일
1  ->  일
2  ->  이
40  ->  사십
1960  ->  천구백육십
12  ->  십이
3  ->  삼
24  ->  이십사
29  ->  이십구
4  ->  사
57  ->  오십칠
4  ->  사
6  ->  육
2014  ->  이천십사
12  ->  십이
06 

11  ->  십일
20  ->  이십
1837  ->  천팔백삼십칠
2  ->  이
16  ->  십육
3  ->  삼
16  ->  십육
3  ->  삼
7  ->  칠
30  ->  삼십
11  ->  십일
11  ->  십일
7  ->  칠
4  ->  사
6000  ->  육천
776  ->  칠백칠십육
11  ->  십일
8  ->  팔
4  ->  사
25  ->  이십오
10  ->  십
14  ->  십사
2  ->  이
2  ->  이
14  ->  십사
2,078  ->  이천칠십팔
1,614  ->  천육백십사
28  ->  이십팔
7  ->  칠
31  ->  삼십일
17  ->  십칠
35  ->  삼십오
292  ->  이백구십이
2011  ->  이천십일
2013  ->  이천십삼
3  ->  삼
6  ->  육
606  ->  육백육
21  ->  이십일
2  ->  이
10  ->  십
3  ->  삼
9  ->  구
6  ->  육
27  ->  이십칠
20  ->  이십
18  ->  십팔
5  ->  오
19  ->  십구
9  ->  구
3  ->  삼
20  ->  이십
4000  ->  사천
1964  ->  천구백육십사
25  ->  이십오
3  ->  삼
1  ->  일
30  ->  삼십
112  ->  백십이
6  ->  육
3  ->  삼
1  ->  일
50  ->  오십
3  ->  삼
13  ->  십삼
2  ->  이
3  ->  삼
11  ->  십일
13  ->  십삼
45  ->  사십오
15  ->  십오
50  ->  오십
10  ->  십
70  ->  칠십
5  ->  오
1  ->  일
4  ->  사
5  ->  오
2015  ->  이천십오
8  ->  팔
4  ->  사
8  ->  팔
1  ->  일
10  ->  십
2007  ->  이천칠
12  ->  십이
30  ->  삼십
2013  ->  이천십삼
580  ->  오백팔십
2017  ->  이천십칠
4  ->  사
100

2016  ->  이천십육
100  ->  백
2017  ->  이천십칠
78  ->  칠십팔
5000  ->  오천
23  ->  이십삼
2017  ->  이천십칠
2  ->  이
1,917  ->  천구백십칠
2016  ->  이천십육
208  ->  이백팔
1  ->  일
3  ->  삼
4  ->  사
2  ->  이
5  ->  오
2008  ->  이천팔
24  ->  이십사
8  ->  팔
27  ->  이십칠
4  ->  사
386  ->  삼백팔십육
27  ->  이십칠
1501  ->  천오백일
1570  ->  천오백칠십
1392  ->  천삼백구십이
1910  ->  천구백십
2003  ->  이천삼
156  ->  백오십육
7  ->  칠
13  ->  십삼
10  ->  십
50  ->  오십
30  ->  삼십
20  ->  이십
12  ->  십이
7  ->  칠
2  ->  이
5  ->  오
1  ->  일
2  ->  이
1  ->  일
2000  ->  이천
6  ->  육
10  ->  십
1  ->  일
2  ->  이
38  ->  삼십팔
90  ->  구십
1  ->  일
3  ->  삼
2  ->  이
2,000  ->  이천
20  ->  이십
26  ->  이십육
5  ->  오
3  ->  삼
2  ->  이
3  ->  삼
1  ->  일
2  ->  이
2  ->  이
3  ->  삼
7  ->  칠
8000  ->  팔천
43  ->  사십삼
3  ->  삼
48  ->  사십팔
93  ->  구십삼
1,995  ->  천구백구십오
1,550  ->  천오백오십
14  ->  십사
1  ->  일
63  ->  육십삼
1000  ->  천
6  ->  육
16  ->  십육
45  ->  사십오
1,200  ->  천이백
1,300  ->  천삼백
4  ->  사
8  ->  팔
1  ->  일
19  ->  십구
1  ->  일
37  ->  삼십칠
15  ->  십오
12  ->  십이
6  ->  육

4  ->  사
31  ->  삼십일
2,800  ->  이천팔백
100  ->  백
74  ->  칠십사
7  ->  칠
500  ->  오백
39  ->  삼십구
2  ->  이
5  ->  오
800  ->  팔백
1  ->  일
24  ->  이십사
2  ->  이
3  ->  삼
1  ->  일
2  ->  이
7  ->  칠
11  ->  십일
20  ->  이십
50  ->  오십
60  ->  육십
54  ->  오십사
3  ->  삼
1  ->  일
2  ->  이
200  ->  이백
400  ->  사백
1  ->  일
19  ->  십구
21  ->  이십일
9  ->  구
10  ->  십
3  ->  삼
4  ->  사
1  ->  일
1  ->  일
12  ->  십이
7  ->  칠
12  ->  십이
4  ->  사
8  ->  팔
24  ->  이십사
0  ->  영
12  ->  십이
1,500  ->  천오백
23  ->  이십삼
29  ->  이십구
2  ->  이
111  ->  백십일
1  ->  일
1  ->  일
25  ->  이십오
300  ->  삼백
2017  ->  이천십칠
10  ->  십
37  ->  삼십칠
8  ->  팔
10  ->  십
2007  ->  이천칠
302  ->  삼백이
9  ->  구
2007  ->  이천칠
10  ->  십
9  ->  구
3  ->  삼
5  ->  오
33  ->  삼십삼
5  ->  오
3  ->  삼
1  ->  일
6  ->  육
27  ->  이십칠
29  ->  이십구
165  ->  백육십오
48  ->  사십팔
3  ->  삼
2  ->  이
2  ->  이
16  ->  십육
18  ->  십팔
3  ->  삼
1  ->  일
8  ->  팔
4  ->  사
1  ->  일
1  ->  일
6  ->  육
2  ->  이
4  ->  사
1  ->  일
4  ->  사
21  ->  이십일
2019  ->  이천십구
2  ->  이
2  ->  이

6  ->  육
21  ->  이십일
2  ->  이
350  ->  삼백오십
100  ->  백
2  ->  이
6  ->  육
7  ->  칠
7  ->  칠
50  ->  오십
5  ->  오
2  ->  이
3000  ->  삼천
2  ->  이
2018  ->  이천십팔
9  ->  구
8  ->  팔
10  ->  십
2  ->  이
29  ->  이십구
1  ->  일
1  ->  일
35  ->  삼십오
3  ->  삼
2  ->  이
5  ->  오
6  ->  육
200  ->  이백
24  ->  이십사
7  ->  칠
400  ->  사백
3  ->  삼
10  ->  십
11  ->  십일
2011  ->  이천십일
3  ->  삼
1,000  ->  천
1  ->  일
1,000  ->  천
66  ->  육십육
7  ->  칠
80  ->  팔십
0  ->  영
18  ->  십팔
1  ->  일
5  ->  오
4  ->  사
95  ->  구십오
19  ->  십구
23  ->  이십삼
111  ->  백십일
169  ->  백육십구
7,000  ->  칠천
26  ->  이십육
26  ->  이십육
10  ->  십
6  ->  육
7  ->  칠
700  ->  칠백
1  ->  일
500  ->  오백
2  ->  이
3  ->  삼
50  ->  오십
2010  ->  이천십
3  ->  삼
6  ->  육
16  ->  십육
9  ->  구
15  ->  십오
9  ->  구
15  ->  십오
20  ->  이십
2013  ->  이천십삼
14  ->  십사
2  ->  이
23  ->  이십삼
7  ->  칠
8  ->  팔
39  ->  삼십구
6  ->  육
1  ->  일
2018  ->  이천십팔
2019  ->  이천십구
25  ->  이십오
25  ->  이십오
3  ->  삼
27  ->  이십칠
13  ->  십삼
10  ->  십
3000  ->  삼천
7  ->  칠
109  ->  백구
9  -> 

...

3  ->  삼
6  ->  육
19  ->  십구
17  ->  십칠
23  ->  이십삼
50  ->  오십
28  ->  이십팔
3  ->  삼
2  ->  이
99  ->  구십구
191  ->  백구십일
7  ->  칠
9  ->  구
0  ->  영
3  ->  삼
1  ->  일
2  ->  이
14  ->  십사
21  ->  이십일
2018  ->  이천십팔
30  ->  삼십
1  ->  일
2  ->  이
1  ->  일
30  ->  삼십
3  ->  삼
1  ->  일
5  ->  오
2019  ->  이천십구
2  ->  이
2  ->  이
800  ->  팔백
250  ->  이백오십
2019  ->  이천십구
2  ->  이
3  ->  삼
7  ->  칠
1  ->  일
6  ->  육
10  ->  십
2  ->  이
30  ->  삼십
25  ->  이십오
2  ->  이
4  ->  사
40  ->  사십
30  ->  삼십
2018  ->  이천십팔
7  ->  칠
19  ->  십구
5  ->  오
2  ->  이
70  ->  칠십
48  ->  사십팔
100  ->  백
6  ->  육
5  ->  오
4  ->  사
3  ->  삼
2  ->  이
1  ->  일
32  ->  삼십이
9  ->  구
4  ->  사
90  ->  구십
10  ->  십
2017  ->  이천십칠
4  ->  사
4  ->  사
20  ->  이십
40  ->  사십
9  ->  구
3300  ->  삼천삼백
37  ->  삼십칠
2016  ->  이천십육
3  ->  삼
20  ->  이십
24  ->  이십사
1  ->  일
62  ->  육십이
26  ->  이십육
11  ->  십일
15  ->  십오
26  ->  이십육
3  ->  삼
100  ->  백
25  ->  이십오
100  ->  백
8  ->  팔
24  ->  이십사
8  ->  팔
27  ->  이십칠
10  ->  십
20  ->  이십
24  ->  이

260  ->  이백육십
28  ->  이십팔
113  ->  백십삼
2  ->  이
152  ->  백오십이
66  ->  육십육
5  ->  오
21  ->  이십일
2  ->  이
0  ->  영
9  ->  구
1  ->  일
24  ->  이십사
22  ->  이십이
3  ->  삼
4  ->  사
3  ->  삼
6  ->  육
146  ->  백사십육
4  ->  사
1  ->  일
14  ->  십사
6  ->  육
30  ->  삼십
4  ->  사
2  ->  이
2  ->  이
14  ->  십사
41  ->  사십일
34  ->  삼십사
2010  ->  이천십
4  ->  사
2006  ->  이천육
9  ->  구
10  ->  십
5  ->  오
5  ->  오
1  ->  일
2  ->  이
300  ->  삼백
200  ->  이백
1930  ->  천구백삼십
14  ->  십사
5841  ->  오천팔백사십일
9,795  ->  구천칠백구십오
139  ->  백삼십구
80  ->  팔십
10  ->  십
25  ->  이십오
10  ->  십
2  ->  이
80  ->  팔십
5  ->  오
158  ->  백오십팔
5  ->  오
2  ->  이
7  ->  칠
69  ->  육십구
600  ->  육백
7  ->  칠
113  ->  백십삼
6  ->  육
6  ->  육
5  ->  오
29  ->  이십구
17  ->  십칠
4  ->  사
38  ->  삼십팔
0  ->  영
78  ->  칠십팔
9  ->  구
3  ->  삼
3  ->  삼
21  ->  이십일
23  ->  이십삼
2019  ->  이천십구
7  ->  칠
2  ->  이
38  ->  삼십팔
90  ->  구십
91  ->  구십일
4  ->  사
10  ->  십
2  ->  이
10  ->  십
70  ->  칠십
2  ->  이
5  ->  오
27  ->  이십칠
100  ->  백
2008  ->  이천팔
2016  ->  이천십육
9

2500  ->  이천오백
21  ->  이십일
14  ->  십사
10  ->  십
22  ->  이십이
9  ->  구
3  ->  삼
400  ->  사백
7  ->  칠
1  ->  일
3  ->  삼
46  ->  사십육
29  ->  이십구
14  ->  십사
4  ->  사
2  ->  이
9,000  ->  구천
1  ->  일
1,000  ->  천
4  ->  사
47  ->  사십칠
3  ->  삼
2  ->  이
2013  ->  이천십삼
3  ->  삼
8  ->  팔
29  ->  이십구
20  ->  이십
1  ->  일
31  ->  삼십일
1  ->  일
4  ->  사
1950  ->  천구백오십
1970  ->  천구백칠십
22  ->  이십이
23  ->  이십삼
2018  ->  이천십팔
27  ->  이십칠
2017  ->  이천십칠
20  ->  이십
2014  ->  이천십사
25  ->  이십오
2017  ->  이천십칠
19  ->  십구
24  ->  이십사
11  ->  십일
8  ->  팔
8  ->  팔
9  ->  구
70  ->  칠십
22  ->  이십이
20  ->  이십
50  ->  오십
1  ->  일
3  ->  삼
7  ->  칠
88  ->  팔십팔
1987  ->  천구백팔십칠
1988  ->  천구백팔십팔
2013  ->  이천십삼
10  ->  십
43  ->  사십삼
1  ->  일
1  ->  일
2  ->  이
2016  ->  이천십육
1  ->  일
100  ->  백
2  ->  이
75  ->  칠십오
300  ->  삼백
2  ->  이
1  ->  일
2  ->  이
6  ->  육
11  ->  십일
10  ->  십
10  ->  십
10  ->  십
1993  ->  천구백구십삼
6  ->  육
12  ->  십이
23  ->  이십삼
12  ->  십이
8  ->  팔
1  ->  일
30  ->  삼십
1  ->  일
96  ->  구십육
3  ->  삼
27

8  ->  팔
18  ->  십팔
16  ->  십육
4510  ->  사천오백십
483  ->  사백팔십삼
9  ->  구
5  ->  오
22  ->  이십이
1  ->  일
2  ->  이
2  ->  이
1  ->  일
180  ->  백팔십
150  ->  백오십
1  ->  일
1  ->  일
21  ->  이십일
12  ->  십이
14  ->  십사
10  ->  십
4  ->  사
1  ->  일
23  ->  이십삼
3  ->  삼
1  ->  일
1  ->  일
1  ->  일
2  ->  이
34  ->  삼십사
33  ->  삼십삼
5  ->  오
32  ->  삼십이
5  ->  오
34  ->  삼십사
66  ->  육십육
204  ->  이백사
114  ->  백십사
318  ->  삼백십팔
10  ->  십
5  ->  오
31  ->  삼십일
30  ->  삼십
25  ->  이십오
2  ->  이
6  ->  육
1  ->  일
30  ->  삼십
60  ->  육십
3  ->  삼
15  ->  십오
1  ->  일
6  ->  육
9  ->  구
3,397  ->  삼천삼백구십칠
68  ->  육십팔
2,323  ->  이천삼백이십삼
2  ->  이
1  ->  일
4  ->  사
4  ->  사
1  ->  일
100  ->  백
7  ->  칠
2017  ->  이천십칠
3  ->  삼
2009  ->  이천구
6  ->  육
5  ->  오
3  ->  삼
11  ->  십일
20  ->  이십
3  ->  삼
1919  ->  천구백십구
3  ->  삼
31  ->  삼십일
10  ->  십
176  ->  백칠십육
3  ->  삼
9  ->  구
10  ->  십
11  ->  십일
5  ->  오
30  ->  삼십
6  ->  육
27  ->  이십칠
40  ->  사십
6  ->  육
13  ->  십삼
737  ->  칠백삼십칠
127  ->  백이십칠
104  ->  백사
88  ->  팔십팔
3  ->

ValueError: invalid literal for int() with base 10: '①'